In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_10.pkl', 'rb') as f:
    DCT_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_20.pkl', 'rb') as f:
    DCT_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_30.pkl', 'rb') as f:
    DCT_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_40.pkl', 'rb') as f:
    DCT_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_50x50_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
DCT_10_normalized = normalize_max_row_norm(DCT_10)
DCT_20_normalized = normalize_max_row_norm(DCT_20)
DCT_30_normalized = normalize_max_row_norm(DCT_30)
DCT_40_normalized = normalize_max_row_norm(DCT_40)

DCT_10_observations = generate_observations(img_dataset, DCT_10_normalized, 0.025)
DCT_20_observations = generate_observations(img_dataset, DCT_20_normalized, 0.025)
DCT_30_observations = generate_observations(img_dataset, DCT_30_normalized, 0.025)
DCT_40_observations = generate_observations(img_dataset, DCT_40_normalized, 0.025)

DCT_10_train, DCT_10_test = split_dataset(DCT_10_observations, train_ratio = 0.9, seed = 0)
DCT_20_train, DCT_20_test = split_dataset(DCT_20_observations, train_ratio = 0.9, seed = 0)
DCT_30_train, DCT_30_test = split_dataset(DCT_30_observations, train_ratio = 0.9, seed = 0)
DCT_40_train, DCT_40_test = split_dataset(DCT_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 10)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'Memory_Residual-DCT-Additive_Depth10-.025/5 Projections'

DCT_10_proj5_weights, DCT_10_proj5_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj5', display=True)

DCT_20_proj5_weights, DCT_20_proj5_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj5', display=True)

DCT_30_proj5_weights, DCT_30_proj5_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj5', display=True)

DCT_40_proj5_weights, DCT_40_proj5_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, DCT_10_proj5_weights, DCT_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, DCT_20_proj5_weights, DCT_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, DCT_30_proj5_weights, DCT_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, DCT_40_proj5_weights, DCT_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.11471334538526005
Average test loss: 0.005014081900732385
Epoch 2/300
Average training loss: 0.02854664697084162
Average test loss: 0.004729208871929182
Epoch 3/300
Average training loss: 0.024939228620794084
Average test loss: 0.004596915629381935
Epoch 4/300
Average training loss: 0.023691738203167917
Average test loss: 0.004417796780044834
Epoch 5/300
Average training loss: 0.023048692178395058
Average test loss: 0.004378911610278818
Epoch 6/300
Average training loss: 0.022664727807044982
Average test loss: 0.004324963355643882
Epoch 7/300
Average training loss: 0.02240383119550016
Average test loss: 0.004312098530845509
Epoch 8/300
Average training loss: 0.022175846444235908
Average test loss: 0.004279693557777338
Epoch 9/300
Average training loss: 0.02199753185113271
Average test loss: 0.004283875272091892
Epoch 10/300
Average training loss: 0.021846761125657293
Average test loss: 0.004240718091941542
Ep

In [8]:
DCT_10_proj5_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj5_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj5_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj5_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj5_psnr = average_PSNR(DCT_10_proj5_model, DCT_10_proj5_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj5_psnr = average_PSNR(DCT_20_proj5_model, DCT_20_proj5_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj5_psnr = average_PSNR(DCT_30_proj5_model, DCT_30_proj5_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj5_psnr = average_PSNR(DCT_40_proj5_model, DCT_40_proj5_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.35
Average PSNR for Projection Layer 1 across 2500 images: 27.65
Average PSNR for Projection Layer 2 across 2500 images: 27.79
Average PSNR for Projection Layer 3 across 2500 images: 27.89
Average PSNR for Projection Layer 4 across 2500 images: 27.94
Average PSNR for Projection Layer 0 across 2500 images: 27.83
Average PSNR for Projection Layer 1 across 2500 images: 28.28
Average PSNR for Projection Layer 2 across 2500 images: 28.55
Average PSNR for Projection Layer 3 across 2500 images: 28.73
Average PSNR for Projection Layer 4 across 2500 images: 28.95
Average PSNR for Projection Layer 0 across 2500 images: 28.50
Average PSNR for Projection Layer 1 across 2500 images: 29.19
Average PSNR for Projection Layer 2 across 2500 images: 29.52
Average PSNR for Projection Layer 3 across 2500 images: 29.86
Average PSNR for Projection Layer 4 across 2500 images: 30.05
Average PSNR for Projection Layer 0 across 2500 images: 29.17
Average 

In [9]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'Memory_Residual-DCT-Additive_Depth10-.025/15 Projections'

DCT_10_proj15_weights, DCT_10_proj15_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj15', display=True)

DCT_20_proj15_weights, DCT_20_proj15_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj15', display=True)

DCT_30_proj15_weights, DCT_30_proj15_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj15', display=True)

DCT_40_proj15_weights, DCT_40_proj15_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, DCT_10_proj15_weights, DCT_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, DCT_20_proj15_weights, DCT_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, DCT_30_proj15_weights, DCT_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, DCT_40_proj15_weights, DCT_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 4.23250624540117
Average test loss: 0.01485995440226462
Epoch 2/300
Average training loss: 1.2993019488652546
Average test loss: 0.005305695115278165
Epoch 3/300
Average training loss: 0.8500322567621866
Average test loss: 0.0047371646463871
Epoch 4/300
Average training loss: 0.5932625586721633
Average test loss: 0.005876512903306219
Epoch 5/300
Average training loss: 0.43336970422003
Average test loss: 0.006212593199892176
Epoch 6/300
Average training loss: 0.33328892869419524
Average test loss: 0.004440141424122784
Epoch 7/300
Average training loss: 0.2701881922483444
Average test loss: 0.004388049233704805
Epoch 8/300
Average training loss: 0.22512941008143955
Average test loss: 0.004337842708246576
Epoch 9/300
Average training loss: 0.19132043443785773
Average test loss: 0.004313141423381037
Epoch 10/300
Average training loss: 0.16558483320474623
Average test loss: 0.004276697126941549
Epoch 11/300
Average 

In [10]:
DCT_10_proj15_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj15_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj15_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj15_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj15_psnr = average_PSNR(DCT_10_proj15_model, DCT_10_proj15_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj15_psnr = average_PSNR(DCT_20_proj15_model, DCT_20_proj15_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj15_psnr = average_PSNR(DCT_30_proj15_model, DCT_30_proj15_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj15_psnr = average_PSNR(DCT_40_proj15_model, DCT_40_proj15_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.35
Average PSNR for Projection Layer 1 across 2500 images: 27.53
Average PSNR for Projection Layer 2 across 2500 images: 27.65
Average PSNR for Projection Layer 3 across 2500 images: 27.66
Average PSNR for Projection Layer 4 across 2500 images: 27.74
Average PSNR for Projection Layer 5 across 2500 images: 27.78
Average PSNR for Projection Layer 6 across 2500 images: 27.77
Average PSNR for Projection Layer 7 across 2500 images: 27.80
Average PSNR for Projection Layer 8 across 2500 images: 27.85
Average PSNR for Projection Layer 9 across 2500 images: 27.88
Average PSNR for Projection Layer 10 across 2500 images: 27.90
Average PSNR for Projection Layer 11 across 2500 images: 27.93
Average PSNR for Projection Layer 12 across 2500 images: 27.96
Average PSNR for Projection Layer 13 across 2500 images: 27.99
Average PSNR for Projection Layer 14 across 2500 images: 28.02
Average PSNR for Projection Layer 0 across 2500 images: 27.63
Ave

In [11]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'Memory_Residual-DCT-Additive_Depth10-.025/30 Projections'

DCT_10_proj30_weights, DCT_10_proj30_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj30', display=True)

DCT_20_proj30_weights, DCT_20_proj30_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj30', display=True)

DCT_30_proj30_weights, DCT_30_proj30_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj30', display=True)

DCT_40_proj30_weights, DCT_40_proj30_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, DCT_10_proj30_weights, DCT_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, DCT_20_proj30_weights, DCT_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, DCT_30_proj30_weights, DCT_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, DCT_40_proj30_weights, DCT_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 32.951022449069555
Average test loss: 0.007526970128218333
Epoch 2/300
Average training loss: 16.002623407999675
Average test loss: 0.005512813715885083
Epoch 3/300
Average training loss: 12.553012383355036
Average test loss: 0.151429182285236
Epoch 4/300
Average training loss: 9.751518324957953
Average test loss: 0.008843458876013757
Epoch 5/300
Average training loss: 8.335578926086425
Average test loss: 0.9092359509401852
Epoch 6/300
Average training loss: 6.032805817074246
Average test loss: 478.3390896566576
Epoch 7/300
Average training loss: 4.958695643107096
Average test loss: 0.23822252231008476
Epoch 8/300
Average training loss: 4.229195634841919
Average test loss: 0.005379078603867027
Epoch 9/300
Average training loss: 3.5607689442104764
Average test loss: 0.007609495194007953
Epoch 10/300
Average training loss: 3.2380026660495336
Average test loss: 0.022673201911151408
Epoch 11/300
Average training lo

In [12]:
DCT_10_proj30_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj30_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj30_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj30_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj30_psnr = average_PSNR(DCT_10_proj30_model, DCT_10_proj30_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj30_psnr = average_PSNR(DCT_20_proj30_model, DCT_20_proj30_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj30_psnr = average_PSNR(DCT_30_proj30_model, DCT_30_proj30_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj30_psnr = average_PSNR(DCT_40_proj30_model, DCT_40_proj30_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.11
Average PSNR for Projection Layer 1 across 2500 images: 27.16
Average PSNR for Projection Layer 2 across 2500 images: 27.38
Average PSNR for Projection Layer 3 across 2500 images: 27.45
Average PSNR for Projection Layer 4 across 2500 images: 27.51
Average PSNR for Projection Layer 5 across 2500 images: 27.55
Average PSNR for Projection Layer 6 across 2500 images: 27.59
Average PSNR for Projection Layer 7 across 2500 images: 27.58
Average PSNR for Projection Layer 8 across 2500 images: 27.59
Average PSNR for Projection Layer 9 across 2500 images: 27.67
Average PSNR for Projection Layer 10 across 2500 images: 27.68
Average PSNR for Projection Layer 11 across 2500 images: 27.68
Average PSNR for Projection Layer 12 across 2500 images: 27.65
Average PSNR for Projection Layer 13 across 2500 images: 27.74
Average PSNR for Projection Layer 14 across 2500 images: 27.77
Average PSNR for Projection Layer 15 across 2500 images: 27.70
Av